In [8]:
from datetime import datetime
from collections import defaultdict

from skyscanner_api import user_share_flight, get_flight_from_airport

In [64]:
users = [
    {"departure_iata": "VIE", "start_date": "2025-06-01", "end_date": "2025-07-30"},
    {"departure_iata": "JFK", "start_date": "2025-06-01", "end_date": "2025-07-30"},
    {"departure_iata": "HEL", "start_date": "2025-06-01", "end_date": "2025-07-30"}
]
res_user_0, places_user_0 = get_flight_from_airport(users[0]["departure_iata"], users[0]["start_date"], users[0]
                                                    ["end_date"], 1)
res_user_1, places_user_1 = get_flight_from_airport(users[1]["departure_iata"], users[1]["start_date"], users[1]
                                                    ["end_date"], 1)
res_user_2, places_user_2 = get_flight_from_airport(users[2]["departure_iata"], users[2]["start_date"], users[2]
                                                    ["end_date"], 1)

print(len(res_user_0))
print(len(res_user_1))
print(len(res_user_2))

122
144
119


In [65]:
list_of_lists = [res_user_0, res_user_1, res_user_2]

In [91]:
from collections import defaultdict
from itertools import product
from datetime import datetime

# helper ─ convert nested date-dict → datetime
def to_dt(d):
    return datetime(d['year'], d['month'], d['day'],
                    d.get('hour', 0), d.get('minute', 0), d.get('second', 0))

def triplet_overlap(lists,
                    dest_key='destination_place_id',
                    start_key='departure_date',
                    end_key='return_date',
                    min_days=5,
                    max_days=14):
    """
    One dict per list is kept when:
      1) all three share the same dest;
      2) [max(start), min(end)] is a valid interval.
    Returns [{'dest': d, 'interval': (start, end), 'triplet': combo}, …]
    """
    n = len(lists)
    buckets = defaultdict(lambda: [[] for _ in range(n)])

    # group by destination
    for idx, lst in enumerate(lists):
        for item in lst:
            buckets[item[dest_key]][idx].append(item)

    res = []
    for dest, per_list in buckets.items():
        if any(not b for b in per_list):
            continue
        for combo in product(*per_list):
            start = max(to_dt(i[start_key]) for i in combo)
            end   = min(to_dt(i[end_key])   for i in combo)
            if start < end and (end - start).days > min_days and (end - start).days < max_days:
                      # positive overlap
                res.append({'dest': dest,
                            'total_days': end - start,
                            'total_price': sum(int(i['price']) for i in combo),
                            'triplet': combo,
                            'interval': (start, end),
                    })
    return res

filtered_lists = triplet_overlap(list_of_lists)

filtered_lists.sort(key=lambda x: x['total_price'])

def format_trip_options(filtered_lists, max_options=5):
    result = []
    for match in filtered_lists[:max_options]:
        trip_info = []
        trip_info.append(f"Destination: {match['triplet'][0]['outbound_place_id']}")
        trip_info.append(f"Total trip duration: {match['total_days'].days} days")
        trip_info.append(f"Trip dates: {match['interval'][0].strftime('%B %d, %Y')} to {match['interval'][1].strftime('%B %d, %Y')}")
        
        for i, user in enumerate(match['triplet']):
            trip_info.append(f"  Departure: {user['departure_date']['month']}/{user['departure_date']['day']}/{user['departure_date']['year']}")
            trip_info.append(f"  Return: {user['return_date']['month']}/{user['return_date']['day']}/{user['return_date']['year']}")
            trip_info.append(f"  Cost: {user['price']} EUR")
            
        trip_info.append(f"Total cost for all users: {sum(int(user['price']) for user in match['triplet'])} EUR")
        result.append('\n'.join(trip_info))
    
    return '\n\n'.join(result)

# Example usage:
formatted_output = format_trip_options(filtered_lists)
print(formatted_output)


Destination: Bucharest Otopeni
Total trip duration: 7 days
Trip dates: June 24, 2025 to July 01, 2025
  Departure: 6/20/2025
  Return: 7/2/2025
  Cost: 32 EUR
  Departure: 6/24/2025
  Return: 7/1/2025
  Cost: 446 EUR
  Departure: 6/17/2025
  Return: 7/5/2025
  Cost: 144 EUR
Total cost for all users: 622 EUR

Destination: Copenhagen
Total trip duration: 6 days
Trip dates: June 25, 2025 to July 01, 2025
  Departure: 6/25/2025
  Return: 7/17/2025
  Cost: 89 EUR
  Departure: 6/24/2025
  Return: 7/1/2025
  Cost: 447 EUR
  Departure: 6/22/2025
  Return: 7/2/2025
  Cost: 144 EUR
Total cost for all users: 680 EUR

Destination: Oslo Gardermoen
Total trip duration: 7 days
Trip dates: June 26, 2025 to July 03, 2025
  Departure: 6/26/2025
  Return: 7/29/2025
  Cost: 150 EUR
  Departure: 6/23/2025
  Return: 7/3/2025
  Cost: 459 EUR
  Departure: 6/25/2025
  Return: 7/15/2025
  Cost: 98 EUR
Total cost for all users: 707 EUR

Destination: Oslo Gardermoen
Total trip duration: 7 days
Trip dates: June 26